In [1]:
# https://dplyr.tidyverse.org/reference/context.html
from datar.all import *

In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

[2021-03-13 00:30:12][datar][   INFO] `summarise()` ungrouping output (override with `_groups` argument)


,g,n
0,a,1
1,b,2
2,c,3


In [3]:
gf >> showme()

[2021-03-13 00:30:12][datar][   INFO] # [DataFrameGroupBy] Groups: ['g'] (3)


,g,x,y
0,a,0.888835,0.614335
1,b,0.983451,0.170819
2,b,0.896694,0.327994
3,c,0.074433,0.917238
4,c,0.807236,0.935654
5,c,0.165058,0.218777


In [4]:
gf >> mutate(id=cur_group_id()) >> showme()

,g,x,y,id
0,a,0.888835,0.614335,0
1,b,0.983451,0.170819,1
2,b,0.896694,0.327994,1
3,c,0.074433,0.917238,2
4,c,0.807236,0.935654,2
5,c,0.165058,0.218777,2


In [5]:
gf >> summarise(row=cur_group_rows()) >> showme()

[2021-03-13 00:30:13][datar][   INFO] `summarise()` regrouping output by ['g']. You can override using the `.groups` argument.
[2021-03-13 00:30:13][datar][   INFO] # [DataFrameGroupBy] Groups: ['g'] (3)


,g,row
0,a,0
1,b,1
2,b,2
3,c,3
4,c,4
5,c,5


In [6]:
gf_group = gf >> summarise(data=[cur_group()])
gf_group 

[2021-03-13 00:30:13][datar][   INFO] `summarise()` ungrouping output (override with `_groups` argument)


,g,data
0,a,g 3 a 4 5
1,b,g 3 b 4 b 5
2,c,g 3 c 4 c 5 c


In [7]:
gf_group >> pull(f.data, to='list')

[   g
 3  a
 4   
 5   ,
    g
 3  b
 4  b
 5   ,
    g
 3  c
 4  c
 5  c]

In [8]:
gf_data = gf >> summarise(data=[cur_data()])
gf_data

[2021-03-13 00:30:13][datar][   INFO] `summarise()` ungrouping output (override with `_groups` argument)


,g,data
0,a,x y 3 0.888835 0.614335 4 ...
1,b,x y 3 0.983451 0.170819 4 ...
2,c,x y 3 0.074433 0.917238 4 ...


In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
 3  0.888835  0.614335
 4                    
 5                    ,
           x         y
 3  0.983451  0.170819
 4  0.896694  0.327994
 5                    ,
           x         y
 3  0.074433  0.917238
 4  0.807236  0.935654
 5  0.165058  0.218777]

In [10]:
gf_data_all = gf >> summarise(data=[cur_data_all()])
gf_data_all

[2021-03-13 00:30:13][datar][   INFO] `summarise()` ungrouping output (override with `_groups` argument)


,g,data
0,a,g x y 3 a 0.888835 0.614...
1,b,g x y 3 b 0.983451 0.170...
2,c,g x y 3 c 0.074433 0.917...


In [11]:
gf_data_all >> pull(f.data, to='list')

[   g         x         y
 3  a  0.888835  0.614335
 4                       
 5                       ,
    g         x         y
 3  b  0.983451  0.170819
 4  b  0.896694  0.327994
 5                       ,
    g         x         y
 3  c  0.074433  0.917238
 4  c  0.807236  0.935654
 5  c  0.165058  0.218777]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

,x,y
0,x 0.7900279243540883,y 0.37740734135777476
1,x 0.9671756699413814,y 0.029179194753120932
2,x 0.8040607874933128,y 0.10758002114779909
3,x 0.0055403010103483846,y 0.8413261815245825
4,x 0.6516299325937484,y 0.8754475349043658
5,x 0.027244204041674373,y 0.047863235194903306


In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

,g,x,y
0,a,x 0.7900279243540883,y 0.37740734135777476
1,b,x 0.9671756699413814,y 0.029179194753120932
2,b,x 0.8040607874933128,y 0.10758002114779909
3,c,x 0.0055403010103483846,y 0.8413261815245825
4,c,x 0.6516299325937484,y 0.8754475349043658
5,c,x 0.027244204041674373,y 0.047863235194903306
